In [264]:
import pandas as pd
from data_service import DataService

eth_ds = DataService("ETH-USDT", "3min")
eth_data = eth_ds.load_market_data()

btc_ds = DataService("BTC-USDT", "3min")
btc_data = btc_ds.load_market_data()
btc_data.columns = [col + "_btc" if col != 'timestamp' else col for col in btc_data.columns]
df = pd.merge(eth_data, btc_data, on='timestamp', how='inner')
display(df.tail(25))

,id,timestamp,open,close,high,low,volume,amount,symbol,timeframe,id_btc,open_btc,close_btc,high_btc,low_btc,volume_btc,amount_btc,symbol_btc,timeframe_btc
499259,499976,1717255080,3803.46,3808.10,3808.11,3803.41,20.622082,78499.070841,ETH-USDT,3min,999260,67759.5,67802.2,67802.3,67755.9,0.798577,54127.041419,BTC-USDT,3min
499260,499977,1717255260,3807.64,3811.21,3812.72,3807.64,20.578292,78433.533825,ETH-USDT,3min,999261,67802.2,67811.1,67811.1,67802.2,0.453593,30758.022476,BTC-USDT,3min
499261,499978,1717255440,3811.21,3809.39,3811.51,3807.84,13.702690,52200.299574,ETH-USDT,3min,999262,67811.0,67771.8,67811.1,67753.0,1.583993,107365.359818,BTC-USDT,3min
499262,499979,1717255620,3809.38,3809.58,3810.61,3809.27,11.864353,45202.881945,ETH-USDT,3min,999263,67771.8,67777.4,67789.6,67771.7,0.665843,45129.477068,BTC-USDT,3min
499263,499980,1717255800,3809.80,3810.96,3811.27,3809.69,8.653332,32974.535327,ETH-USDT,3min,999264,67777.4,67787.6,67792.4,67777.3,0.324551,22000.264253,BTC-USDT,3min
499264,499981,1717255980,3810.98,3810.34,3811.19,3809.81,12.340830,47028.877487,ETH-USDT,3min,999265,67787.6,67778.8,67787.6,67777.2,0.605628,41052.035265,BTC-USDT,3min
499265,499982,1717256160,3810.31,3809.65,3810.31,3808.13,15.949818,60755.144507,ETH-USDT,3min,999266,67778.7,67777.3,67778.8,67764.4,0.468913,31778.354657,BTC-USDT,3min
499266,499983,1717256340,3809.94,3810.02,3810.02,3808.50,11.768041,44831.643012,ETH-USDT,3min,999267,67778.8,67793.0,67794.2,67778.6,2.814567,190777.111993,BTC-USDT,3min
499267,499984,1717256520,3810.02,3809.72,3810.02,3808.51,15.065405,57387.912737,ETH-USDT,3min,999268,67793.0,67788.7,67793.0,67788.6,0.563869,38224.447058,BTC-USDT,3min
499268,499985,1717256700,3809.71,3807.10,3809.71,3805.79,21.915795,83428.549511,ETH-USDT,3min,999269,67788.7,67764.1,67788.7,67753.5,1.081997,73327.750989,BTC-USDT,3min


In [265]:
df = df.drop(['id', 'symbol', 'id_btc', 'symbol_btc', 'timeframe_btc', 'timeframe'], axis=1)
display(df)

,timestamp,open,close,high,low,volume,amount,open_btc,close_btc,high_btc,low_btc,volume_btc,amount_btc
0,1627388460,2269.54,2273.66,2275.24,2269.10,202.926697,461190.490193,37921.8,37993.0,38005.0,37921.7,12.430279,472068.436372
1,1627388640,2273.35,2272.92,2277.08,2271.57,70.050489,159372.123758,37992.9,37981.3,38069.1,37953.0,7.152814,271949.237565
2,1627388820,2272.18,2272.46,2275.39,2271.00,30.162256,68543.216371,37976.0,37989.6,38015.3,37963.3,5.819570,221064.718963
3,1627389000,2272.04,2277.76,2281.25,2270.61,80.891352,184225.258406,37990.9,38081.1,38148.0,37956.4,14.301592,544326.172536
4,1627389180,2277.76,2288.81,2289.50,2277.69,233.528651,533500.292762,38080.4,38252.7,38260.0,38074.8,21.509973,821417.451089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499279,1717258680,3797.02,3797.31,3798.36,3794.15,33.293991,126396.428656,67661.9,67651.6,67661.9,67617.4,0.992349,67116.679561
499280,1717258860,3797.65,3798.00,3800.91,3797.65,31.610752,120110.356425,67651.5,67697.4,67702.7,67651.5,1.257906,85138.234531
499281,1717259040,3798.34,3801.87,3801.87,3797.31,16.853405,64026.894622,67695.2,67707.2,67707.2,67680.8,0.399566,27047.434768
499282,1717259220,3801.86,3804.10,3804.59,3801.86,10.593988,40287.996050,67707.2,67713.6,67713.6,67707.1,1.059070,71711.811459


In [266]:
import cross_validation

cross_val_count = 3
train_segment_size = 300000
val_segment_size = 50000
test_segment_size = 50000
cross_val_segmentation_map = cross_validation.create_segmentation_map(cross_val_count, train_segment_size, val_segment_size, test_segment_size)
display(cross_val_segmentation_map)

{'total_data_needed': 500000,
 'train_segment_0': (0, 300000),
 'val_segment_0': (300000, 350000),
 'test_segment_0': (350000, 400000),
 'evaluation_segment_0': (300000, 400000),
 'train_segment_1': (50000, 350000),
 'val_segment_1': (350000, 400000),
 'test_segment_1': (400000, 450000),
 'evaluation_segment_1': (350000, 450000),
 'train_segment_2': (100000, 400000),
 'val_segment_2': (400000, 450000),
 'test_segment_2': (450000, 500000),
 'evaluation_segment_2': (400000, 500000)}

In [267]:
df = df.tail(cross_val_segmentation_map['total_data_needed']).reset_index(drop=True)
display(df)

,timestamp,open,close,high,low,volume,amount,open_btc,close_btc,high_btc,low_btc,volume_btc,amount_btc
0,1627388460,2269.54,2273.66,2275.24,2269.10,202.926697,461190.490193,37921.8,37993.0,38005.0,37921.7,12.430279,472068.436372
1,1627388640,2273.35,2272.92,2277.08,2271.57,70.050489,159372.123758,37992.9,37981.3,38069.1,37953.0,7.152814,271949.237565
2,1627388820,2272.18,2272.46,2275.39,2271.00,30.162256,68543.216371,37976.0,37989.6,38015.3,37963.3,5.819570,221064.718963
3,1627389000,2272.04,2277.76,2281.25,2270.61,80.891352,184225.258406,37990.9,38081.1,38148.0,37956.4,14.301592,544326.172536
4,1627389180,2277.76,2288.81,2289.50,2277.69,233.528651,533500.292762,38080.4,38252.7,38260.0,38074.8,21.509973,821417.451089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499279,1717258680,3797.02,3797.31,3798.36,3794.15,33.293991,126396.428656,67661.9,67651.6,67661.9,67617.4,0.992349,67116.679561
499280,1717258860,3797.65,3798.00,3800.91,3797.65,31.610752,120110.356425,67651.5,67697.4,67702.7,67651.5,1.257906,85138.234531
499281,1717259040,3798.34,3801.87,3801.87,3797.31,16.853405,64026.894622,67695.2,67707.2,67707.2,67680.8,0.399566,27047.434768
499282,1717259220,3801.86,3804.10,3804.59,3801.86,10.593988,40287.996050,67707.2,67713.6,67713.6,67707.1,1.059070,71711.811459


In [268]:
segments = {}
for n in range(cross_val_count):
    cross_val_cycle_keys = [key for key in cross_val_segmentation_map.keys() if str(n) in key]
    
    segments[n] = {}
    for key in cross_val_cycle_keys:
        start_end_tuple = cross_val_segmentation_map[key]
        segment = cross_validation.get_segment(df, *start_end_tuple)
        segments[n][key] = segment

In [269]:
cross_val_set_n = 2
cross_val_set = segments[cross_val_set_n]
train_data = cross_val_set[f'train_segment_{cross_val_set_n}']
val_data = cross_val_set[f'val_segment_{cross_val_set_n}']
test_data = cross_val_set[f'test_segment_{cross_val_set_n}']

train_data = train_data.drop(['timestamp'], axis=1)
val_data = val_data.drop(['timestamp'], axis=1)
test_data = test_data.drop(['timestamp'], axis=1)

In [270]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from labeler import BinaryWinFinder

win_direction, candle_span, distance_threshold = "long", 40, 0.01

steps = np.linspace(5, 250, 10)
steps = [int(n) for n in steps]


def create_labels(df):
    win_finder = BinaryWinFinder(df, win_direction, candle_span, distance_threshold)
    df['wins'] = win_finder.find_wins()
    return df


def get_scaler(scaler_fit_data):
    scaler = RobustScaler()
    scaler = scaler.fit(scaler_fit_data)
    return scaler


def ema(df, span, target_col):
    closes = df[target_col]
    ema = closes.ewm(span=span).mean()
    label = f"{target_col}_ema_{span}"
    return ema.rename(label)


def apply_emas(df):
    df = df.copy()
    for n in steps:
        series = ema(df, n, 'close')
        df[series.name] = series
        series = ema(df, n, 'close_btc')
        df[series.name] = series
    return df


def log_series_if_ascending(df):
    for column in df.columns:
        df[column] = np.log(df[column])
        print(f"Log transformation applied on column '{column}'")
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df


def get_X_and_y(df):
    y = df['wins'].reset_index(drop=True)
    X = df.drop(['wins'], axis=1).reset_index(drop=True)
    return X, y


def atr(df, period=14):
    high_column = df['high']
    low_column = df['low']
    prev_close_column = df['close'].shift(-1)
    high_low_arr = high_column - low_column
    high_close_arr = abs(high_column - prev_close_column)
    low_close_arr = abs(low_column - prev_close_column)
    tr = pd.concat([high_low_arr, high_close_arr, low_close_arr], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/period, adjust=False).mean()
    label = f"atr{period}"
    return atr.rename(label)


def obv(df):
    copy = df.copy()
    obv = (np.sign(copy["close"].diff()) * copy["volume"]).fillna(0).rolling(250).sum()
    return obv


def process(df, scaler):
    df = create_labels(df)
    
    df = apply_emas(df)
    # df['atr'] = atr(df)
    # df['obv'] = obv(df)
    
    cols_not_win = [col for col in df.columns if col != "wins"]
    cols_not_win_not_ema = [col for col in cols_not_win if "ema" not in col]
    # df[cols_not_win_not_ema] = log_series_if_ascending(df[cols_not_win_not_ema])
    df[cols_not_win_not_ema] = df[cols_not_win_not_ema].diff()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    # df[cols_not_win] = scaler.transform(df[cols_not_win])


    X, y = get_X_and_y(df)
    return X, y

In [271]:
train_data_copy = train_data.copy()
train_data_copy = apply_emas(train_data.copy())
# train_data_with_emas['atr'] = atr(train_data_with_emas)
# train_data_with_emas['obv'] = obv(train_data_with_emas)
cols_not_win = [col for col in train_data.columns if col != "wins"]
cols_not_win_not_ema = [col for col in cols_not_win if "ema" not in col]

# train_data_copy[cols_not_win_not_ema] = log_series_if_ascending(train_data_copy[cols_not_win_not_ema])
train_data_copy[cols_not_win_not_ema] = train_data_copy[cols_not_win_not_ema].diff()
train_data_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
train_data_copy.dropna(inplace=True)
scaler = get_scaler(train_data_copy)
X_train, y_train = process(train_data, scaler)
X_val, y_val = process(val_data, scaler)
X_test, y_test = process(test_data, scaler)


In [272]:
display(X_train)

,open,close,high,low,volume,amount,open_btc,close_btc,high_btc,low_btc,...,close_ema_141,close_btc_ema_141,close_ema_168,close_btc_ema_168,close_ema_195,close_btc_ema_195,close_ema_222,close_btc_ema_222,close_ema_250,close_btc_ema_250
0,0.29,0.02,-1.47,-3.25,-31.744094,-83758.339138,3.7,5.5,-19.7,-28.1,...,2633.310071,38314.169504,2633.310060,38314.166369,2633.310051,38314.164103,2633.310045,38314.162387,2633.310040,38314.161000
1,-0.38,-7.75,-0.59,-6.82,113.759464,298699.666766,-0.8,-39.9,-0.6,-26.9,...,2630.693365,38301.603514,2630.699265,38301.629803,2630.703527,38301.648796,2630.706752,38301.663159,2630.709359,38301.674772
2,-7.75,-0.82,-4.13,-0.03,-68.507169,-180357.444723,-39.9,-4.1,-18.2,2.5,...,2629.175762,38294.274241,2629.185284,38294.318588,2629.192170,38294.350648,2629.197380,38294.374906,2629.201594,38294.394526
3,-0.82,1.79,-0.31,1.32,-42.755481,-112124.305591,-4.0,30.6,17.8,14.0,...,2628.633550,38296.172106,2628.643556,38296.198850,2628.650799,38296.218246,2628.656285,38296.232956,2628.660724,38296.244874
4,1.79,2.95,0.74,2.52,17.753930,46779.280727,30.6,24.6,11.6,30.7,...,2628.781401,38301.683892,2628.788862,38301.675197,2628.794277,38301.669045,2628.798385,38301.664466,2628.801716,38301.660810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299994,3.37,-3.53,0.59,-1.73,62.516077,118246.434826,44.0,-50.3,26.8,-46.9,...,1880.520103,35027.060219,1880.075502,34995.550230,1879.960777,34973.119366,1880.046285,34957.008294,1880.259530,34945.017547
299995,-3.53,-2.37,-3.67,-2.37,-12.545787,-24101.916189,-50.4,-77.0,-72.7,-68.9,...,1880.688834,35032.573455,1880.222537,35000.555553,1880.088728,34977.664066,1880.157978,34961.147233,1880.357063,34948.790316
299996,-2.39,0.99,-1.83,0.04,6.227769,11675.696342,-77.0,-8.2,-80.9,-39.1,...,1880.869132,35037.893547,1880.379549,35005.404600,1880.225476,34982.078718,1880.277547,34965.175509,1880.461708,34952.467684
299997,1.00,-1.25,0.40,1.25,-105.244599,-199059.397543,-2.0,-42.0,-2.6,17.4,...,1881.029285,35042.547159,1880.519909,35009.699220,1880.348073,34986.019752,1880.384834,34968.790975,1880.555559,34955.781088


In [273]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.model_selection import train_test_split


def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'alpha': trial.suggest_float('alpha', 0.1, 1.0),
        'lambda': trial.suggest_float('lambda', 0.1, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'early_stopping_rounds': 10
    }
    
    model = XGBClassifier(**param, use_label_encoder=False, eval_metric='logloss')
    
    # Train the model on the training set
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    # Make predictions on the validation set
    y_proba = model.predict_proba(X_val)[:, 1]
    
    threshold = 0.55 # Adjust this threshold value as needed
    y_pred = (y_proba >= threshold).astype(int)
    
    # Evaluate the model on the validation set for the positive class (class 1)
    precision = precision_score(y_val, y_pred, pos_label=1)
    recall = recall_score(y_val, y_pred, pos_label=1)
    
    # Custom metric that combines precision and recall smoothly
    score = precision * 0.7 + recall * 0.3
    
    return score


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

[I 2024-06-07 01:01:41,982] A new study created in memory with name: no-name-da55fe63-c76d-4249-ad4e-743ea17bdb28
[I 2024-06-07 01:01:42,547] Trial 0 finished with value: 0.15895266887959053 and parameters: {'n_estimators': 131, 'max_depth': 6, 'learning_rate': 0.06747465617187133, 'subsample': 0.9593792953031579, 'colsample_bytree': 0.7993171395838892, 'alpha': 0.22434421141756034, 'lambda': 0.17845009866880499, 'scale_pos_weight': 4.687401901816944}. Best is trial 0 with value: 0.15895266887959053.
c:\Users\rusta\.virtualenvs\hft-2GmTtn1u\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-06-07 01:01:42,877] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 181, 'max_depth': 3, 'learning_rate': 0.11682881833959297, 'subsample': 

In [274]:
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report

# Initialize the model with the best hyperparameters
best_params = study.best_params
best_params['use_label_encoder'] = False
best_params['eval_metric'] = 'logloss'
best_params['early_stopping_rounds'] = 10

model = XGBClassifier(**best_params)

# Train the model
model.fit(X_train, y_train , eval_set=[(X_val, y_val)], verbose=False)

# Make predictions
y_proba = model.predict_proba(X_test)[:, 1]
    
threshold = 0.55 # Adjust this threshold value as needed
y_pred = (y_proba >= threshold).astype(int)

# Evaluate the model
precision = precision_score(y_test, y_pred, pos_label=1)
print(f"Precision: {precision:.2f}")

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)




Precision: 0.21
Confusion Matrix:
[[    0 38957]
 [    0 10326]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     38957
           1       0.21      1.00      0.35     10326

    accuracy                           0.21     49283
   macro avg       0.10      0.50      0.17     49283
weighted avg       0.04      0.21      0.07     49283



c:\Users\rusta\.virtualenvs\hft-2GmTtn1u\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rusta\.virtualenvs\hft-2GmTtn1u\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rusta\.virtualenvs\hft-2GmTtn1u\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [275]:
first_100_proba = model.predict_proba(X_val[:3000])[:, 1]
first_100_pred = (first_100_proba >= threshold).astype(int)
first_100_y = y_val[:3000]
print(first_100_y.value_counts(normalize=True))

precision = precision_score(first_100_y, first_100_pred, pos_label=1)
print(f"Precision: {precision:.2f}")

conf_matrix = confusion_matrix(first_100_y, first_100_pred)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(first_100_y, first_100_pred)
print("Classification Report:")
print(class_report)

wins
0    0.868667
1    0.131333
Name: proportion, dtype: float64
Precision: 0.13
Confusion Matrix:
[[   0 2606]
 [   0  394]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2606
           1       0.13      1.00      0.23       394

    accuracy                           0.13      3000
   macro avg       0.07      0.50      0.12      3000
weighted avg       0.02      0.13      0.03      3000



c:\Users\rusta\.virtualenvs\hft-2GmTtn1u\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rusta\.virtualenvs\hft-2GmTtn1u\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rusta\.virtualenvs\hft-2GmTtn1u\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r